In [8]:
#
# Work in progress to port to contemporary python tools.
# November 2020
#
# Reference: https://github.com/mbernico/CS570.git
#

from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import sklearn.model_selection as model_selection
# from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import pickle
#conda install requests will be needed
import requests, json

### Building a simple model

In [9]:
# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [10]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [12]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)

In [13]:
rfc = RandomForestClassifier(n_estimators=100, n_jobs=2)

In [14]:
rfc.fit(X_train, y_train)

RandomForestClassifier(n_jobs=2)

In [15]:
print("Accuracy = %0.2f" % accuracy_score (y_test, rfc.predict(X_test)))
print(classification_report(y_test, rfc.predict(X_test)))

Accuracy = 0.89
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.73      1.00      0.85        11
           2       1.00      0.69      0.82        13

    accuracy                           0.89        38
   macro avg       0.91      0.90      0.89        38
weighted avg       0.92      0.89      0.89        38



### Model serialization / marshalling

In [16]:
pickle.dump(rfc, open("iris_rfc.pkl", "wb"))

In [17]:
my_random_forest = pickle.load(open("iris_rfc.pkl", "rb"))

In [18]:
my_random_forest

RandomForestClassifier(n_jobs=2)

In [19]:
print(classification_report(y_test, my_random_forest.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.73      1.00      0.85        11
           2       1.00      0.69      0.82        13

    accuracy                           0.89        38
   macro avg       0.91      0.90      0.89        38
weighted avg       0.92      0.89      0.89        38



#### Starting the model server

In a python terminal outside of Jupyter, run the following command:

```
$ export FLASK_APP=model-server.py
$ python flask run
```
Example output:

```
 * Serving Flask app "model-server.py"
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 ```

In [22]:
#
# Make a prediction by calling the REST API.
#

url = "http://localhost:5000/api"
data = json.dumps({'sl':5.84,'sw':3.0, 'pl':3.75,'pw':1.1}) 
r = requests.post(url, data)

print(r.json())

{'results': {'y_hat': 1}}
